## 简答题
1. **模型投票法（Voting）**
   如果你训练了五个不同的模型，并且它们都达到了 95% 的精度，可以尝试将它们组合成一个投票集成（voting ensemble），通常能得到更好的结果。

   * 当模型差异很大时（例如 SVM 分类器、决策树分类器、逻辑回归分类器等），效果更好。
   * 如果它们训练在不同的训练实例上，效果会更佳（这正是 bagging 和 pasting 的目的）。
   * 即使不是这样，只要模型之间差异足够大，集成仍然有效。

2. **硬投票 vs. 软投票**

   * 硬投票分类器统计每个分类器的投票，选择得票最多的类别。
   * 软投票分类器计算每个类别的平均预测概率，选择概率最高的类别。
   * 软投票会给高置信度的预测更大权重，通常表现更好，但前提是每个分类器都能估计类别概率（例如在 Scikit-Learn 中使用 SVM 分类器时必须设置 `probability=True`）。

3. **加速训练**

   * 对于 Bagging 集成，可以将训练分配到多个服务器，因为每个预测器都是独立的。
   * 同样的道理适用于 Pasting 集成和随机森林。
   * 但 Boosting 中每个预测器依赖前一个预测器，训练是顺序进行的，因此无法并行。
   * Stacking 集成中的每一层预测器可以并行训练，但必须等上一层的预测器训练完毕后才能继续。

4. **包外评估（Out-of-Bag Evaluation, OOB）**

   * 在 Bagging 集成中，每个预测器都会用到未训练过的实例进行评估（这些样本被“留出”）。
   * 这提供了对集成结果相对无偏的评估，不需要额外验证集。

5. **随机森林与 Extra-Trees**

   * 在随机森林中，节点划分时只考虑特征的随机子集。
   * Extra-Trees 更进一步：它们使用完全随机的阈值，而不是搜索最优阈值。
   * 这种随机性类似正则化，可以减少过拟合。
   * 如果随机森林过拟合，Extra-Trees 可能表现更好。
   * 它们训练速度更快，但在性能上可能不一定比随机森林更好或更差。

6. **AdaBoost 欠拟合的解决办法**

   * 如果 AdaBoost 欠拟合，可以尝试：

     * 增加估计器数量
     * 减小基学习器的正则化
     * 适度增加学习率

7. **梯度提升（Gradient Boosting）过拟合的解决办法**

   * 如果梯度提升过拟合，可以尝试降低学习率。
   * 也可以使用早停法，在找到最佳预测器数量时提前终止训练。


## 编程题
1. 加载MNIST数据集（**8_sklearn做分类** 里有介绍），将其分为一个训练集、一个验证集和一个测试集（例如使用40000个实例训练，10000个实例验证，最后20000个实例测试）。然后训练多个分类器，比如一个随机森林分类器、一个极端随机树分类器和一个SVM。接下来，尝试使用软投票法或者硬投票法将它们组合成一个集成，这个集成在验证集上的表现要胜过它们各自单独的表现。成功找到集成后，在测试集上测试。与单个的分类器相比，它的性能要好多少？

In [1]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

mnist = fetch_openml('mnist_784', as_frame=False, parser="auto")
X,y = mnist.data,mnist.target
X_train, X_valid, X_test, y_train, y_valid, y_test = X[:40000], X[40000:50000],X[50000:], y[:40000], y[40000:50000], y[50000:]

In [4]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.svm import SVC
vot = VotingClassifier(estimators=[
    ('rf', RandomForestClassifier(n_estimators=300,max_depth=5,random_state=42)),
    ("svc", SVC(random_state=42,C=0.05,)),
    ("et",ExtraTreesClassifier(max_depth=5,random_state=42)),
], voting='hard',n_jobs=-1)
vot.fit(X_train, y_train)

VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(max_depth=5,
                                                     n_estimators=300,
                                                     random_state=42)),
                             ('svc', SVC(C=0.05, random_state=42)),
                             ('et',
                              ExtraTreesClassifier(max_depth=5,
                                                   random_state=42))],
                 n_jobs=-1)

In [5]:
for name, clf in vot.named_estimators_.items():
    print(name, " valid accuracy =", clf.score(X_valid, y_valid.astype(int)), "train accuracy =", clf.score(X_train, y_train.astype(int)), "test accuracy =", clf.score(X_test, y_test.astype(int)))

print(vot.score(X_valid,y_valid))
print(vot.score(X_test,y_test))

rf  valid accuracy = 0.861 train accuracy = 0.8639 test accuracy = 0.87445
svc  valid accuracy = 0.9378 train accuracy = 0.9455 test accuracy = 0.94835
et  valid accuracy = 0.8295 train accuracy = 0.83365 test accuracy = 0.84475
0.8767
0.89185



2. 运行上一题中的单个分类器，用验证集进行预测，然后用预测结果创建一个新的训练集：新训练集中的每个实例都是一个向量，这个向量包含所有分类器对于一个图像的一组预测，目标值是图像的类。

- 恭喜，你成功训练了一个混合器，结合第一层的分类器，它们一起构成了一个堆叠集成。现在在测试集上评估这个集成。对于测试集中的每个图像，使用所有的分类器进行预测，然后将预测结果提供给混合器，得到集成的预测。与前面训练的投票分类器相比，这个集成的结果如何？现在再次尝试使用StackingClassifier。你得到了更好的性能吗？如果是这样，为什么？

In [6]:
import numpy as np
X_valid_predict = np.empty((len(X_valid), len(vot.estimators_)))
for i, name_clf in enumerate(vot.named_estimators_.items()):
    _, clf = name_clf
    X_valid_predict[:, i] = clf.predict(X_valid)

final_estimator = RandomForestClassifier(random_state=43)
final_estimator.fit(X_valid_predict, y_valid)

RandomForestClassifier(random_state=43)

In [7]:
final_estimator.score(X_valid_predict, y_valid)

0.9427

In [8]:
# 在测试集评估 堆叠分类器的性能
X_test_predict = np.empty((len(X_test), len(vot.estimators_)))
for i, name_clf in enumerate(vot.named_estimators_.items()):
    _, clf = name_clf
    X_test_predict[:, i] = clf.predict(X_test)

final_estimator.score(X_test_predict, y_test)

0.94505

In [11]:
from sklearn.svm import LinearSVC
from sklearn.ensemble import StackingClassifier
import warnings
from sklearn.exceptions import ConvergenceWarning

# 忽略收敛警告（不推荐，但可以暂时使用）
warnings.filterwarnings("ignore", category=ConvergenceWarning)
# 现在来评估 内置的 StackingClassifier， 内置的StackingClassifier用交叉验证预测，所以不需要手动分 训练+验证集
X_train_full, y_train_full = np.vstack((X_train, X_valid)), np.concatenate((y_train, y_valid))
stacking_clf = StackingClassifier(
     estimators = [("rf", RandomForestClassifier(n_estimators=300, max_leaf_nodes=32, random_state=42)),
                  ("et", ExtraTreesClassifier(n_estimators=300, max_leaf_nodes=32, random_state=43)),
                  ("svc", LinearSVC(random_state=42, C=0.05, dual=True))],
    final_estimator = RandomForestClassifier(random_state=43),
    cv = 5
)

stacking_clf.fit(X_train_full, y_train_full)

StackingClassifier(cv=5,
                   estimators=[('rf',
                                RandomForestClassifier(max_leaf_nodes=32,
                                                       n_estimators=300,
                                                       random_state=42)),
                               ('et',
                                ExtraTreesClassifier(max_leaf_nodes=32,
                                                     n_estimators=300,
                                                     random_state=43)),
                               ('svc',
                                LinearSVC(C=0.05, dual=True, random_state=42))],
                   final_estimator=RandomForestClassifier(random_state=43))

In [12]:
stacking_clf.score(X_test, y_test)

0.9369